In [ ]:
#여기는 만약 필요하다면!!
"""
!pip install ultralytics
!pip install onnxruntime-gpu

from ultralytics import YOLO
import zipfile
import cv2
import os
from PIL import Image

# Load the YOLOv8 model
model = YOLO('yolov8n.pt')
model.to('cuda')
# Export the model to ONNX format
model.export(format='onnx')  # creates 'yolov8n.onnx'

# Load the exported ONNX model
onnx_model = YOLO('yolov8n.onnx')
"""

In [ ]:
#제가 헷갈려서 dataset_name을 이미지 이름으로 하고 save_path를 저장폴더이름으로 되게 해놨어요
#save_paht의 dataset_name폴더에 이미지가 저장되게 고쳐야하는데 아직 안 고쳐진 상태입니다!!
def yolo_vid(dataset_name: str, video_path: str, dataset_save_path: str):
    cap = cv2.VideoCapture(video_path)
    assert cap.isOpened(), "Error reading video file"
    
    absolute_paths = []
    idx = 0

    crop_dir_name = dataset_save_path
    if not os.path.exists(crop_dir_name):
        os.mkdir(crop_dir_name)

    frame_count = 0 
    while cap.isOpened():
        success, im0 = cap.read()
        if not success:
            break
            
            
        if frame_count%50 == 0 :
            results = onnx_model(im0, classes=0, conf = 0.4)
            boxes = results[0].boxes.xyxy.tolist()  # Filter for person class

            for box in boxes:
                idx += 1
                crop_obj = im0[int(box[1]):int(box[3]), int(box[0]):int(box[2])]
                cv2.imwrite(os.path.join(crop_dir_name, f"{dataset_name}_{idx}.png"), crop_obj)
                absolute_path = os.path.abspath(f"{dataset_save_path}/{dataset_name}_{idx}.png")
                absolute_paths.append(absolute_path)
        frame_count+=1
    cap.release()
    cv2.destroyAllWindows()
    return absolute_paths


def yolo_zip(dataset_name: str, zip_file_path: str, dataset_save_path: str):
    absolute_paths = []

    crop_dir_name = dataset_save_path
    if not os.path.exists(crop_dir_name):
        os.mkdir(crop_dir_name)
    
    if not os.path.exists('zip_extract'):
        os.makedirs('zip_extract')
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall('zip_extract')
    image_paths = []

    for root, _, files in os.walk('zip_extract'):
        for file in files:
            if file.lower().endswith(('.png', '.jpg', '.jpeg')):
                absolute_path = os.path.abspath(os.path.join(root, file))
                image_paths.append(absolute_path)   
    
    frame_count = 0 
    for imgpath in image_paths:
        im0 = Image.open(imgpath)
        if frame_count%1 == 0 :
            results = onnx_model(im0, classes=0, conf = 0.4)
            boxes = results[0].boxes.xyxy.tolist()  # Filter for person class
            
            for idx, box in enumerate(results[0].boxes.xyxy.tolist()):
                x1, y1, x2, y2 = box
                img_cropped = im0.crop((x1,y1,x2,y2))
                img_cropped.save(f"{dataset_save_path}/{dataset_name}_{idx}.png")
                absolute_path = os.path.abspath(f"{dataset_save_path}/{dataset_name}_{idx}.png")
                absolute_paths.append(absolute_path)
        frame_count+=1
        
    return absolute_paths